In [ ]:
import keras

In [ ]:
from keras.models import Sequential

In [ ]:
from keras.layers import LSTM, Dense, Dropout

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
from keras.utils import np_utils

Now that we've imported everything we need form Keras, we're all set to go!